## Question 1

#### <ins>Equation A</ins>
The equation $x+y-z=0$ is valid in a linear program because it is a linear equation.

#### <ins>Equation B</ins>
The equation $x \le \frac{100}{y}$ is not valid in a linear program because it is a non-linear function.


#### <ins>Equation C</ins>
The equation $3x+2y \le \sqrt{5}$ is valid in a linear program because it is a linear function.


#### <ins>Equation D</ins>
The equation $\sqrt{5}x+2y = 50$ is valid in a linear program because it is a linear function.


#### <ins>Equation E</ins>
The equation $\sqrt{5x}+10y = 100$ is not valid in a linear program because it is a non-linear function as a result of $\sqrt{5x}$.


#### <ins>Equation F</ins>
The equation $x^{2}+y^{2} \ge 45$ is not valid in a linear program because it contains quadratic terms.


## Question 6

In [39]:
import gurobipy as gp

# Create a new model
model = gp.Model("cutting_stock")

# Orders: [Width, Number of Rolls]
orders = [(5, 150), (7, 200), (9, 300)]

# Generate all combinations to cut 20-foot rolls into widths of 5, 7, and 9 feet
possible_cuts = []
for cut_5 in range(5):
    for cut_7 in range(3):
        for cut_9 in range(3):
            if cut_5 * 5 + cut_7 * 7 + cut_9 * 9 <= 20:
                possible_cuts.append((cut_5, cut_7, cut_9))

# Decision Variables
x = {}
for i, cut in enumerate(possible_cuts):
    x[i] = model.addVar(vtype=gp.GRB.INTEGER, name=f"x_{cut}")

# Objective Function: Minimize the total number of 20-foot rolls
model.setObjective(sum(x[i] for i in x), gp.GRB.MINIMIZE)

# Constraints: Meet the total demand for each order
for j, (width, demand) in enumerate(orders):
    model.addConstr(
        sum(x[i] * possible_cuts[i][j] for i in range(len(possible_cuts))) >= demand, 
        name=f"demand_{j}"
    )

# Solve the model
model.optimize()

# Output Results
print("\nOptimal Solution:")
for v in model.getVars():
    if v.x > 0:  # Only show cuts that are actually used
        print(f"{v.varName}: {v.x}")
print(f"Minimum number of 20-foot rolls needed: {model.objVal}")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i9-10850K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 3 rows, 15 columns and 20 nonzeros
Model fingerprint: 0xab337bf2
Variable types: 0 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 3e+02]
Found heuristic solution: objective 500.0000000
Presolve removed 0 rows and 9 columns
Presolve time: 0.00s
Presolved: 3 rows, 6 columns, 10 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: objective 2.625000e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  262.50000    0    1  500

In [11]:
import gurobipy as gp

model = gp.Model('Cutting_Stock')

# Create the decision variables
a = model.addVar(vtype=gp.GRB.INTEGER, name='orders_1')
b = model.addVar(vtype=gp.GRB.INTEGER, name='orders_2')
c = model.addVar(vtype=gp.GRB.INTEGER, name='orders_3')

combinations = generate_combinations()
combo_vars = {}
for i, combo in enumerate(combinations):
    var_name = f'roll_{i+1}'
    combo_vars[var_name] = model.addVar(vtype=gp.GRB.INTEGER, name=var_name)

# Order constraints
width_5 = model.addConstr(a >= 150, 'width_5')
width_7 = model.addConstr(b >= 200, 'width_7')
width_9 = model.addConstr(c >= 300, 'width_9')

#

# Create the objective function
model.setObjective(5 * a + 7 * b + 9 * c, gp.GRB.MINIMIZE)

# Extra material. You don't need them, but it's nice to know what they do!
model.write(model.ModelName + '.lp')
model.setParam('OutputFlag', 0)

# Solve the model
model.optimize()

# Check the solution
for v in model.getVars():
    print('%s : %g' % (v.varName, v.x))

print(f"Number of Rolls Used : {int(model.objVal // 20 + 1)}")

# How to get Shadow Price
for constraint in model.getConstrs():
    print("--------\n", constraint.ConstrName, "\nRemaining Slack ", constraint.Slack)

orders_1 : 150
orders_2 : 200
orders_3 : 300
num_5 : -0
num_7 : -0
num_9 : -0
Number of Rolls Used : 243
--------
 roll 
Remaining Slack  20.0
--------
 width_5 
Remaining Slack  -0.0
--------
 width_7 
Remaining Slack  -0.0
--------
 width_9 
Remaining Slack  -0.0
